In [9]:
import sympy as sp

t, theta = sp.symbols("t, theta", real = True)

px, py, pz = sp.symbols("px, py, pz", real=True) 

tx0, ty0, tz0 = sp.symbols("tx0, ty0, tz0", real=True) 
tx1, ty1, tz1 = sp.symbols("tx1, ty1, tz1", real=True) 

qx0, qy0, qz0, qw0 = sp.symbols("qx0, qy0, qz0, qw0", real=True) 
qx1, qy1, qz1, qw1 = sp.symbols("qx1, qy1, qz1, qw1", real=True) 

s000, s001, s002, s010, s011, s012, s020, s021, s022 = sp.symbols("s000, s001, s002, s010, s011, s012, s020, s021, s022", real=True)
s100, s101, s102, s110, s111, s112, s120, s121, s122 = sp.symbols("s100, s101, s102, s110, s111, s112, s120, s121, s122", real=True)

def lerp(v0,v1,t):
    return v0*(1-t)+v1*t

# assuming that q1 is qperp = normalize(q1-q2*cosTheta)
def slerp(q0, q1, t):
    return q0 * sp.cos(t*theta) + q1 * sp.sin(t*theta)

def to_rotation_matrix(q):
    return sp.Matrix([[q.a*q.a + q.b*q.b - q.c*q.c - q.d*q.d, 2*(q.b*q.c - q.a*q.d),                 2*(q.b*q.d + q.a*q.c),                 0], 
                      [2*(q.b*q.c + q.a*q.d),                 q.a*q.a - q.b*q.b + q.c*q.c - q.d*q.d, 2*(q.c*q.d - q.a*q.b),                 0], 
                      [2*(q.b*q.d - q.a*q.c),                 2*(q.c*q.d + q.a*q.b),                 q.a*q.a - q.b*q.b - q.c*q.c + q.d*q.d, 0],
                      [0,                                     0,                                     0,                                     1]])

q0 = sp.Quaternion(qw0, qx0, qy0, qz0)
q1 = sp.Quaternion(qw1, qx1, qy1, qz1)
qt = slerp(q0, q1, t)
S0 = sp.Matrix([[s000, s001, s002, 0],
                [s010, s011, s012, 0],
                [s020, s021, s022, 0],
                [   0,    0,    0, 1]])
S1 = sp.Matrix([[s100, s101, s102, 0],
                [s110, s111, s112, 0],
                [s120, s121, s122, 0],
                [   0,    0,    0, 1]])
St = lerp(S0, S1, t)
D0 = sp.Matrix([[1, 0, 0, tx0],
                [0, 1, 0, ty0],
                [0, 0, 1, tz0],
                [0, 0, 0,  1]])
D1 = sp.Matrix([[1, 0, 0, tx1],
                [0, 1, 0, ty1],
                [0, 0, 1, tz1],
                [0, 0, 0,  1]])
Dt = lerp(D0, D1, t)
p = sp.Matrix([px, py, pz, 1])


M = Dt*to_rotation_matrix(qt)*St*p
m0 = M[0]

In [10]:
dm0 = sp.diff(m0, t)
dm0 = sp.expand(dm0)

dm0 = dm0.subs(sp.sin(t*theta)*sp.sin(t*theta),(1-sp.cos(2*t*theta))/2) # remove sin(t*theta)^2
dm0 = dm0.subs(sp.cos(t*theta)*sp.cos(t*theta),(1+sp.cos(2*t*theta))/2) # remove cos(t*theta)^2
dm0 = dm0.subs(sp.sin(t*theta)*sp.cos(t*theta),sp.sin(2*t*theta)/2)     # remove sin(t*theta)*cos(t*theta)
dm0 = sp.expand(dm0)

dm0_cos_sin = sp.collect(dm0, (sp.sin(2*t*theta), sp.cos(2*t*theta)), evaluate=False)

coeff_const = dm0_cos_sin[1]
coeff_sin   = dm0_cos_sin[sp.sin(2*t*theta)]
coeff_cos   = dm0_cos_sin[sp.cos(2*t*theta)]

coeff_sin_t = sp.collect(coeff_sin, t, evaluate=False)
coeff_cos_t = sp.collect(coeff_cos, t, evaluate=False)
c0 = coeff_const
c1 = coeff_cos_t[1]
c2 = coeff_cos_t[t]
c3 = coeff_sin_t[1]
c4 = coeff_sin_t[t]

In [11]:
kc0 = sp.collect(c0, (px, py, pz), evaluate=False)
kc0 = [kc0[1], kc0[px], kc0[py], kc0[pz]]
for i in range(4):
    kc0[i] = sp.expand(kc0[i])
    kc0[i] = kc0[i].subs(qw0*qw0, 1-qx0*qx0-qy0*qy0-qz0*qz0)
    kc0[i] = kc0[i].subs(qw1*qw1, 1-qx1*qx1-qy1*qy1-qz1*qz1)
    kc0[i] = sp.simplify(kc0[i])
kc0[1]

-qw0*qy0*s020 + qw0*qy0*s120 + qw0*qz0*s010 - qw0*qz0*s110 - qw1*qy1*s020 + qw1*qy1*s120 + qw1*qz1*s010 - qw1*qz1*s110 - qx0*qy0*s010 + qx0*qy0*s110 - qx0*qz0*s020 + qx0*qz0*s120 - qx1*qy1*s010 + qx1*qy1*s110 - qx1*qz1*s020 + qx1*qz1*s120 + qy0**2*s000 - qy0**2*s100 + qy1**2*s000 - qy1**2*s100 + qz0**2*s000 - qz0**2*s100 + qz1**2*s000 - qz1**2*s100 - s000 + s100

In [12]:
kc1 = sp.collect(c1, (px, py, pz), evaluate=False)
kc1 = [0, kc1[px], kc1[py], kc1[pz]]
for i in range(4):
    kc1[i] = sp.expand(kc1[i])
    kc1[i] = kc1[i].subs(qw0*qw0, 1-qx0*qx0-qy0*qy0-qz0*qz0)
    kc1[i] = kc1[i].subs(qw1*qw1, 1-qx1*qx1-qy1*qy1-qz1*qz1)
    kc1[i] = sp.simplify(kc1[i])
kc1[3]

2*qw0*qw1*s002*theta - qw0*qy0*s022 + qw0*qy0*s122 + 2*qw0*qy1*s022*theta + qw0*qz0*s012 - qw0*qz0*s112 - 2*qw0*qz1*s012*theta + 2*qw1*qy0*s022*theta + qw1*qy1*s022 - qw1*qy1*s122 - 2*qw1*qz0*s012*theta - qw1*qz1*s012 + qw1*qz1*s112 + 2*qx0*qx1*s002*theta - qx0*qy0*s012 + qx0*qy0*s112 + 2*qx0*qy1*s012*theta - qx0*qz0*s022 + qx0*qz0*s122 + 2*qx0*qz1*s022*theta + 2*qx1*qy0*s012*theta + qx1*qy1*s012 - qx1*qy1*s112 + 2*qx1*qz0*s022*theta + qx1*qz1*s022 - qx1*qz1*s122 + qy0**2*s002 - qy0**2*s102 - 2*qy0*qy1*s002*theta - qy1**2*s002 + qy1**2*s102 + qz0**2*s002 - qz0**2*s102 - 2*qz0*qz1*s002*theta - qz1**2*s002 + qz1**2*s102